# Procesamiento de Lenguaje Natural I

**Autor:** Gonzalo G. Fernandez

Clase 3: Modelos secuenciales - RNNs y LSTM

## Consigna desafío 3
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.

**Sugerencias:**

- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.

In [1]:
from utils import parse_srt_to_dialogue

import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import (
    SimpleRNN,
    Dense,
    Dense,
    LSTM,
    GRU,
    Dropout,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

import keras_tuner as kt

2025-06-20 17:16:40.624193: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-20 17:16:40.748547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750432600.806588    8596 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750432600.821101    8596 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750432600.923159    8596 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## Resolución

Se utilizarán los diálogos de la película argentina "Nueve Reinas" del año 2000 como corpus.

In [7]:
dialogues = parse_srt_to_dialogue("data/nueve_reinas-subtitles.srt")
print(f"Total dialogues extracted: {len(dialogues)}")
for d in dialogues[:5]:
    print(d)
corpus = " ".join(dialogues).lower()

Total dialogues extracted: 1371
¿Qué estás leyendo?
Nada... disculpáme. No, está todo bien.
¿Nada más que esto? Sí.
Esta máquina me vuelve loca. Después lo registro.
1.25 más 3.75 son... 5


### Preprocesamiento

Tokenización de caracteres.

In [8]:
chars = sorted(list(set(corpus)))
char2idx = {char: idx for idx, char in enumerate(chars)}
idx2char = {idx: char for char, idx in char2idx.items()}
vocab_size = len(chars)

tokenized_corpus = [char2idx[c] for c in corpus]

Creación de secuencias de entrada y labels.

In [9]:
seq_length = 100
step = 1

sequences = []
next_chars = []

for i in range(0, len(tokenized_corpus) - seq_length, step):
    sequences.append(tokenized_corpus[i : i + seq_length])
    next_chars.append(tokenized_corpus[i + seq_length])

X = np.array(sequences)
y = np.array(next_chars)

- División del dataset en entrenamiento y validación.
- One-Hot encoding de las secuencias de entrada y los labels.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

X_train_encoded = to_categorical(X_train, num_classes=vocab_size)
y_train_encoded = to_categorical(y_train, num_classes=vocab_size)

X_test_encoded = to_categorical(X_test, num_classes=vocab_size)
y_test_encoded = to_categorical(y_test, num_classes=vocab_size)

Se utiliza como métrica de desempeño la perplejidad.

In [12]:
def perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    return K.exp(K.mean(cross_entropy))

### Creación y entrenamiento de modelo SimpleRNN

In [16]:
model_simple_rnn = Sequential(
    [
        SimpleRNN(128, input_shape=(seq_length, vocab_size)),
        Dense(vocab_size, activation="softmax"),
    ]
)

model_simple_rnn.compile(
    loss="categorical_crossentropy",
    optimizer=RMSprop(learning_rate=0.001),
    metrics=["accuracy", perplexity],
)

model_simple_rnn.fit(
    X_train_encoded,
    y_train_encoded,
    validation_data=(X_test_encoded, y_test_encoded),
    batch_size=64,
    epochs=15,
)

Epoch 1/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.2402 - loss: 2.8092 - perplexity: 17.6803 - val_accuracy: 0.3320 - val_loss: 2.2948 - val_perplexity: 10.0416
Epoch 2/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.3394 - loss: 2.2232 - perplexity: 9.3766 - val_accuracy: 0.3637 - val_loss: 2.1440 - val_perplexity: 8.6480
Epoch 3/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.3672 - loss: 2.0904 - perplexity: 8.1897 - val_accuracy: 0.3755 - val_loss: 2.0915 - val_perplexity: 8.2094
Epoch 4/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.3819 - loss: 2.0320 - perplexity: 7.7285 - val_accuracy: 0.3747 - val_loss: 2.0655 - val_perplexity: 7.9849
Epoch 5/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.3959 - loss: 1.9913 - perplexity: 7.4332 - val_accuracy: 0.3847 - val_loss: 2.0363 - val_perplexity: 7.7799
Epoch 6/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.4023 - loss: 1.9396 - perplexity: 7.0504 - val_accuracy:

- Dado que el vocabulario es de 54, una perplejidad de ~6 indica relativamente buen desempeño.
- El entrenamiento del modelo es relativamente rápido.

### Creación y entrenamiento de modelo LSTM

In [17]:
model_lstm = Sequential(
    [
        LSTM(128, input_shape=(seq_length, vocab_size)),
        Dense(vocab_size, activation="softmax"),
    ]
)

model_lstm.compile(
    loss="categorical_crossentropy",
    optimizer=RMSprop(learning_rate=0.001),
    metrics=["accuracy", perplexity],
)

model_lstm.fit(
    X_train_encoded,
    y_train_encoded,
    validation_data=(X_test_encoded, y_test_encoded),
    batch_size=64,
    epochs=15,
)

Epoch 1/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 27s 41ms/step - accuracy: 0.2075 - loss: 2.9605 - perplexity: 20.4029 - val_accuracy: 0.2896 - val_loss: 2.4689 - val_perplexity: 11.9628
Epoch 2/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 26s 39ms/step - accuracy: 0.3198 - loss: 2.3277 - perplexity: 10.3874 - val_accuracy: 0.3313 - val_loss: 2.2385 - val_perplexity: 9.4906
Epoch 3/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 25s 39ms/step - accuracy: 0.3455 - loss: 2.1858 - perplexity: 9.0339 - val_accuracy: 0.3509 - val_loss: 2.1439 - val_perplexity: 8.6428
Epoch 4/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 25s 39ms/step - accuracy: 0.3710 - loss: 2.0884 - perplexity: 8.1645 - val_accuracy: 0.3758 - val_loss: 2.0705 - val_perplexity: 8.0342
Epoch 5/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 26s 40ms/step - accuracy: 0.3845 - loss: 2.0253 - perplexity: 7.6876 - val_accuracy: 0.3880 - val_loss: 2.0149 - val_perplexity: 7.6031
Epoch 6/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 27s 41ms/step - accuracy: 0.3997 - loss: 1.9735 - perplexity: 7.2882 - val_ac

- Puede observarse como el tiempo de entrenamiento es mucho mayor que con SimpleRNN.

### Creación y entrenamiento de modelo GRU (Gated Recurrent Unit)
En búsqueda de un mejor desempeño que con LSTM.

In [18]:
model_gru = Sequential(
    [
        GRU(128, input_shape=(seq_length, vocab_size)),
        Dense(vocab_size, activation="softmax"),
    ]
)

model_gru.compile(
    loss="categorical_crossentropy",
    optimizer=RMSprop(learning_rate=0.001),
    metrics=["accuracy", perplexity],
)

model_gru.fit(
    X_train_encoded,
    y_train_encoded,
    validation_data=(X_test_encoded, y_test_encoded),
    batch_size=64,
    epochs=15,
)

Epoch 1/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 27s 41ms/step - accuracy: 0.2348 - loss: 2.8658 - perplexity: 19.3814 - val_accuracy: 0.3297 - val_loss: 2.2547 - val_perplexity: 9.6526
Epoch 2/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 27s 41ms/step - accuracy: 0.3442 - loss: 2.1802 - perplexity: 8.9676 - val_accuracy: 0.3694 - val_loss: 2.1006 - val_perplexity: 8.2633
Epoch 3/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 27s 41ms/step - accuracy: 0.3752 - loss: 2.0611 - perplexity: 7.9687 - val_accuracy: 0.3900 - val_loss: 2.0094 - val_perplexity: 7.5660
Epoch 4/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 26s 41ms/step - accuracy: 0.3974 - loss: 1.9626 - perplexity: 7.2159 - val_accuracy: 0.4085 - val_loss: 1.9404 - val_perplexity: 7.0548
Epoch 5/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 27s 41ms/step - accuracy: 0.4194 - loss: 1.9016 - perplexity: 6.7905 - val_accuracy: 0.4262 - val_loss: 1.8978 - val_perplexity: 6.7648
Epoch 6/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 27s 41ms/step - accuracy: 0.4359 - loss: 1.8239 - perplexity: 6.2855 - val_accu

### Comparación de perplejidad de modelos

In [19]:
test_loss, test_acc, test_perplexity = model_simple_rnn.evaluate(X_test_encoded, y_test_encoded, verbose=0)
print(f"SimpleRNN Test Perplexity: {test_perplexity:.4f} (vocab_size={vocab_size})")

test_loss, test_acc, test_perplexity = model_lstm.evaluate(X_test_encoded, y_test_encoded, verbose=0)
print(f"LSTM Test Perplexity: {test_perplexity:.4f} (vocab_size={vocab_size})")

test_loss, test_acc, test_perplexity = model_gru.evaluate(X_test_encoded, y_test_encoded, verbose=0)
print(f"GRU Test Perplexity: {test_perplexity:.4f} (vocab_size={vocab_size})")

SimpleRNN Test Perplexity: 6.7452 (vocab_size=54)
LSTM Test Perplexity: 5.9821 (vocab_size=54)
GRU Test Perplexity: 5.5973 (vocab_size=54)


Ordenando los modelos de forma decreciente utilizando como métrica la perplejidad en el subset de test se obtiene la siguiente lista:

1. GRU
2. LSTM
3. SimpleRNN

Es importante destacar que solo observando la perplejidad el desempeño es muy similar y por lo tanto resalta SimpleRNN como un modelo facil de entrenar que ofrece buenos resultados.

### Búsqueda de hiper parámetros para los modelos en estudio

Implementación de función constructora del modelo

In [2]:
def model_builder(hp):
    model = Sequential()
    rnn_type = hp.Choice("rnn_type", ["SimpleRNN", "LSTM", "GRU"])
    units = hp.Int("units", min_value=64, max_value=256, step=64)
    dropout = hp.Float("dropout", 0.0, 0.3, step=0.1)
    learning_rate = hp.Choice("lr", [1e-2, 1e-3, 1e-4])

    RNN = getattr(tf.keras.layers, rnn_type)
    model.add(RNN(units, input_shape=(seq_length, vocab_size)))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(vocab_size, activation="softmax"))

    model.compile(
        optimizer=RMSprop(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=[perplexity],
    )
    return model

Definición del tuner para realizar búsquedas o para cargar parámetros ya exportados.

In [4]:
tuner = kt.Hyperband(
    model_builder,
    objective=kt.Objective("val_perplexity", direction="min"),
    max_epochs=10,
    directory="output",
    project_name="9reinas_rnn_tuning",
)

Reloading Tuner from output/9reinas_rnn_tuning/tuner0.json


Búsqueda de hiperparámetros.

In [3]:
tuner.search(
    X_train_encoded,
    y_train_encoded,
    validation_data=(X_test_encoded, y_test_encoded),
    epochs=10,
    batch_size=64,
)

NameError: name 'tuner' is not defined

Opcional: Carga de datos ya exportados (útil ante desconexión del notebook).

In [5]:
tuner.reload()

Obtención de mejores hiperparámetros en la búsqueda:

In [14]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
print("Best hyperparameters:")
for hp_name in best_hps.values:
    print(f"{hp_name}: {best_hps.get(hp_name)}")

Best hyperparameters:
rnn_type: LSTM
units: 192
dropout: 0.1
lr: 0.01
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012


Entrenamiento del modelo obtenido:

In [15]:
model = tuner.hypermodel.build(best_hps)

model.fit(
    X_train_encoded,
    y_train_encoded,
    validation_data=(X_test_encoded, y_test_encoded),
    epochs=15,
)

/home/dell-1/workspace/ceia/ceia-nlp1/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/15


I0000 00:00:1750433030.945221   11633 cuda_dnn.cc:529] Loaded cuDNN version 90300


1295/1295 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 2.4636 - perplexity: 13.5686 - val_loss: 1.9168 - val_perplexity: 7.0168
Epoch 2/15
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 1.8361 - perplexity: 6.4843 - val_loss: 1.7499 - val_perplexity: 5.9827
Epoch 3/15
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 1.6494 - perplexity: 5.3809 - val_loss: 1.6686 - val_perplexity: 5.5137
Epoch 4/15
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 1.5465 - perplexity: 4.8519 - val_loss: 1.6254 - val_perplexity: 5.2860
Epoch 5/15
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 1.4651 - perplexity: 4.4722 - val_loss: 1.6054 - val_perplexity: 5.1878
Epoch 6/15
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 1.4013 - perplexity: 4.2120 - val_loss: 1.6024 - val_perplexity: 5.1761
Epoch 7/15
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 1.3535 - perplexity: 4.0078 - val_loss: 1.6036 - val_perplexity: 5.1842
Epoch 8/15
1295/1295 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 1.3263 - p

### Generación de texto

Implementación de greedy search para generación de texto.

In [16]:
def generate_text_greedy(model, seed, length=200):
    seed = seed.lower()
    generated = seed

    for _ in range(length):
        input_seq = [char2idx.get(c, 0) for c in seed[-seq_length:]]
        input_seq = to_categorical([input_seq], num_classes=vocab_size)

        preds = model.predict(input_seq, verbose=0)[0]

        next_idx = np.argmax(preds) # greedy
        next_char = idx2char[next_idx]

        generated += next_char
        seed += next_char

    return generated

Implementación de beam search para generación de texto.

In [17]:
def generate_text_beam_search(model, seed, length=200, beam_width=3):
    seed = seed.lower()
    sequences = [(seed, 0.0)]

    for _ in range(length):
        all_candidates = []

        for seq, score in sequences:
            input_seq = [char2idx.get(c, 0) for c in seq[-seq_length:]]
            input_seq = to_categorical([input_seq], num_classes=vocab_size)
            preds = model.predict(input_seq, verbose=0)[0]

            top_indices = np.argsort(preds)[-beam_width:] # beam search

            for idx in top_indices:
                next_char = idx2char[idx]
                prob = preds[idx]
                candidate = (seq + next_char, score + np.log(prob + 1e-8))  # log-sum
                all_candidates.append(candidate)

        sequences = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)[:beam_width]

    return sequences[0][0]

Comparación de greedy search para generación de texto entre modelos:

In [24]:
print("SimpleRNN:", generate_text_greedy(model_simple_rnn, "la verdad es que ", length=50))
print("LSTM:", generate_text_beam_search(model_lstm, "la verdad es que ", length=50))
print("GRU:", generate_text_beam_search(model_gru, "la verdad es que ", length=50))

SimpleRNN: la verdad es que las para vas a la mi rengunte de mi harmano. ¿no e
LSTM: la verdad es que estás los estampillas. ¿qué querés? no, esto es un
GRU: la verdad es que no tengo que tengo que lo que tengo que lo que te 


Comparación de beam search para generación de texto entre modelos:

In [25]:
print("SimpleRNN:", generate_text_beam_search(model_simple_rnn, "la verdad es que ", length=50))
print("LSTM:", generate_text_beam_search(model_lstm, "la verdad es que ", length=50))
print("GRU:", generate_text_beam_search(model_gru, "la verdad es que ", length=50))

SimpleRNN: la verdad es que tengo que tengo que tengo que tengo que tengo que 
LSTM: la verdad es que estás los estampillas. ¿qué querés? no, esto es un
GRU: la verdad es que no tengo que tengo que lo que tengo que lo que te 


Puede observarse cómo las palabras se generan en su mayoría correctamente con los tres modelos pero en todos los casos la oración generada carece de contenido semántico. Además, hay una tendencia a generar palabras de gran frecuancia (comunes) en el corpus.

## Ejecución del mejor modelo obtenido
Generación de texto con el mejor modelo de los estudiados, resultado de la búsqueda de hiperparámetros.

In [21]:
print("LSTM:", generate_text_beam_search(model, "mira que yo ", length=50))
print("LSTM:", generate_text_beam_search(model, "tomamos algo ", length=50))

LSTM: mira que yo vende la plata. ¿quién es? ¿tía? escucheme, hablam
LSTM: tomamos algo de mierda. ¿quién es? ¿tía? escucheme, hablamos es
